In [1]:
import torch
from torch import nn
from torchvision import transforms, models, datasets
from torch.utils.data import DataLoader

device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else "cpu"
device = "cpu"

In [ ]:
transform = transforms.Compose([
    transforms.Resize(227),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # from ImageNet
])

train_loader = DataLoader(datasets.ImageFolder("cats_and_dogs_filtered/train", transform=transform))
validation_loader = DataLoader(datasets.ImageFolder("cats_and_dogs_filtered/validation", transform=transform))

alexnet = models.alexnet().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(alexnet.parameters(), lr=0.1)

# Train
alexnet.train()
for epoch in range(1, 5 + 1):
    print(f"Epoch {epoch} of 5")
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        predictions = alexnet(images)
        loss = loss_fn(predictions, labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Evaluate
alexnet.eval()
correct = 0

with torch.no_grad():
    for images, labels in validation_loader:
        predictions = alexnet(images).argmax(dim=1)
        correct += (predictions == labels).sum().item()

accuracy = correct / len(validation_loader.dataset)
print(f"Accuracy {accuracy}")